## Packages

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime
from matplotlib.ticker import MaxNLocator


from IPython.display import display

## Settings

In [ ]:
# Source data file
sPath = 'details.txt'

## Import the data

In [ ]:
# import raw data
rawdata = pd.read_csv(sPath, sep='\t', encoding='latin-1')

# replace NaN values with 0
rawdata.fillna(0, inplace=True)

In [ ]:
display(rawdata)

## Data processing

In [ ]:
# print(rawdata.columns)

# Assign each column to a variable
ora = rawdata[rawdata.columns[0]]
disp = rawdata[rawdata.columns[1]]
# serie = rawdata[rawdata.columns[2]]
potenza = rawdata[rawdata.columns[3]]
# energia = rawdata[rawdata.columns[4]]
# temperatura = rawdata[rawdata.columns[5]]
# tensione1 = rawdata[rawdata.columns[6]]
# corrente1 = rawdata[rawdata.columns[7]]
# tensione2 = rawdata[rawdata.columns[8]]
# corrente2 = rawdata[rawdata.columns[9]]
# tensione3 = rawdata[rawdata.columns[10]]
# corrente3 = rawdata[rawdata.columns[11]]
# stato = rawdata[rawdata.columns[12]]
# modulicc = rawdata[rawdata.columns[13]]
# errore = rawdata[rawdata.columns[14]]
# pannello = rawdata[rawdata.columns[15]]
# limite = rawdata[rawdata.columns[16]]
# fase = rawdata[rawdata.columns[17]]
# cosphi = rawdata[rawdata.columns[18]]

# ottieni il mese relativo al log
month = ora[0][:8]

# ottieni le date non ripetute
# per provare i primi due giorni sostituisci 'ora' con 'ora[:3500]'
uniqueTimes = ora.unique()

# rimuovi l'informazione sulla data e sui secondi
uniqueSimpleTime = pd.Series(map(lambda x: x.split(' ')[1][:-3], uniqueTimes))

# ricava il giorno dalla stringa del tempo
days = pd.Series(map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").day, uniqueTimes))
# ottieni i giorni non ripetuti
uniqueDays = days.unique()

# ottieni la potenza totale dei 13 pannelli
totPower = pd.DataFrame(map(lambda time: sum(potenza[ora==time]), uniqueTimes))


## EXPERIMENTING WITH PIVOT TABLES
# first split the time column in two:
processedData = rawdata.copy()

temp = processedData.columns.to_list()
temp[0] = 'Data'
processedData.columns = temp
# convert the date-string column into a date/time object
processedData['Data'] = pd.to_datetime(processedData['Data'])
# create a new column containing the time only (HH:MM)
processedData['Ora'] = processedData['Data'].dt.strftime("%H:%M")
# and reformat 'Date' to keep only the information about the day
processedData['Data'] = processedData['Data'].dt.strftime("%Y-%m-%d")

# list with all the dates (can be used as a filter later)
# uniqueDates = processedData.Data.unique()

# obtain column strings from position
sNDevice = processedData.columns[1]
sPower = processedData.columns[3]

# power per device
stackedFrame = processedData.set_index(['Data', sNDevice, 'Ora'])
stackedFrame.sort_index(inplace=True)

# total power
groupedFrame = processedData.groupby(['Data','Ora'])[sPower].sum().to_frame()

## Plots config

In [ ]:
# dimensione base delle
sz = 10

# dimensione delle etichette x-y, del titolo e dei numeri dei grafici
labelsize = 20
titlesize = 25
numsize = 15

# Plots

## Cumulative daily plots

In [ ]:
for sDate, dayTable in groupedFrame.groupby(level=0):
    # remove date index
    dayTable = dayTable.droplevel(level=0)
    
    fig, ax = plt.subplots(1, 1, figsize=(sz, sz))
    ax.plot(dayTable/1000)
    
    # 
    ax.xaxis.set_major_locator(MaxNLocator(nbins=24))
    ax.grid(b=True)
    ax.set_title(sDate + " - Potenza totale", size = titlesize)
    ax.set_ylabel("Potenza totale [kW]", size = labelsize)
    ax.set_xlabel("Tempo", size = labelsize)
    
    plt.xticks(rotation=45, size = numsize)
    plt.yticks(size = numsize)

## Inverters daily plots

In [ ]:
# Draw one plot for every day
for sDate, dayTable in stackedFrame.groupby(level=0):
    fig, ax = plt.subplots(1, 1, figsize=(sz, sz))
    
    # remove date index
    dayTable = dayTable.droplevel(level=0)
    
    # draw one series for every device
    for sDevice, deviceTable in dayTable.groupby(level=0):
        # remove device index and get power column
        deviceTable = deviceTable.droplevel(level=0)[sPower]/1000
        # plot nth device power
        plt.plot(deviceTable, label=sDevice)
    
    
    ax.xaxis.set_major_locator(MaxNLocator(nbins=24))
    ax.grid(b=True)
    ax.set_title(sDate + " - Potenza per inverter", size = titlesize)
    ax.set_ylabel("Potenza totale [kW]", size = labelsize)
    ax.set_xlabel("Tempo", size = labelsize)
    
    plt.xticks(rotation=45, size = numsize)
    plt.yticks(size = numsize)
    
    # increase current axis by 5%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 1.05, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title = "Inverter")
    